In [2]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM(), simulation=True)
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(.1)

    time.sleep(3)     
        
    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:59607
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:45535
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:36419
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:39029
INFO:root:('127.0.0.1', 45535) Connected to 127.0.0.1:59607
INFO:root:('127.0.0.1', 59607) Conexión aceptada con 127.0.0.1:45535
INFO:root:('127.0.0.1', 36419) Connected to 127.0.0.1:45535
INFO:root:('127.0.0.1', 45535) Conexión aceptada con 127.0.0.1:36419
INFO:root:('127.0.0.1', 59607) Connected to 127.0.0.1:36419
INFO:root:('127.0.0.1', 36419) Conexión aceptada con 127.0.0.1:59607
INFO:root:('127.0.0.1', 39029) Connected to 127.0.0.1:36419
INFO:root:('127.0.0.1', 36419) Conexión aceptada con 127.0.0.1:39029
INFO:root:('127.0.0.1', 45535) Connected to 127.0.0.1:39029
INFO:root:('127.0.0.1', 59607) Connected to 127.0.0.1:39029
INFO:root:('127.0.0.1', 39029) Conexión aceptada con 127

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

ERROR:root:Something went wrong with pytorch lightning. cannot import name 'Popen' from partially initialized module 'multiprocessing.popen_fork' (most likely due to a circular import) (/usr/lib/python3.9/multiprocessing/popen_fork.py)
INFO:root:(('127.0.0.1', 36419)) Training...


Sanity Checking: 0it [00:00, ?it/s]

...

INFO:root:(('127.0.0.1', 45535)) Waiting other nodes.
INFO:root:(('127.0.0.1', 36419)) Round 2 of 2 finished.
INFO:root:(('127.0.0.1', 45535)) Round 2 of 2 finished.
INFO:root:(('127.0.0.1', 36419)) Evaluating...
INFO:root:(('127.0.0.1', 45535)) Evaluating...
INFO:root:(('127.0.0.1', 39029)) Waiting other nodes.
INFO:root:(('127.0.0.1', 39029)) Round 2 of 2 finished.
INFO:root:(('127.0.0.1', 39029)) Evaluating...


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 36419)) Evaluating...
INFO:root:(('127.0.0.1', 59607)) Evaluated. Losss: 0.08113337308168411, Metric: 0.9749000072479248. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 59607)) Finish!!.
INFO:root:(('127.0.0.1', 36419)) Evaluated. Losss: 0.08113337308168411, Metric: 0.9749000072479248. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 36419)) Finish!!.
INFO:root:(('127.0.0.1', 45535)) Evaluated. Losss: 0.08113337308168411, Metric: 0.9749000072479248. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 45535)) Finish!!.
INFO:root:(('127.0.0.1', 39029)) Evaluated. Losss: 0.08113337308168411, Metric: 0.9749000072479248. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 39029)) Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 59607 y desconectándose de 3 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 59607)
DEBUG:root:Closed connection: ('127.0.0.1', 59607)
DEBUG:root:Closed connection: ('127.0.0.1', 5960

In [1]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(0.3)
    print("STPIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIING")
    #nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:55927
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:51551
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:48497
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:52265
INFO:root:('127.0.0.1', 51551) Connected to 127.0.0.1:55927
INFO:root:('127.0.0.1', 55927) Conexión aceptada con 127.0.0.1:51551
INFO:root:('127.0.0.1', 48497) Connected to 127.0.0.1:51551
INFO:root:('127.0.0.1', 51551) Conexión aceptada con 127.0.0.1:48497
INFO:root:('127.0.0.1', 55927) Connected to 127.0.0.1:48497
INFO:root:('127.0.0.1', 48497) Conexión aceptada con 127.0.0.1:55927
INFO:root:('127.0.0.1', 52265) Connected to 127.0.0.1:48497
INFO:root:('127.0.0.1', 48497) Conexión aceptada con 127.0.0.1:52265
INFO:root:('127.0.0.1', 55927) Connected to 127.0.0.1:52265
INFO:root:('127.0.0.1', 51551) Connected to 127.0.0.1:52265
INFO:root:('127.0.0.1', 52265) Conexión aceptada con 127

INFO:root:(('127.0.0.1', 51551)) Evaluating...
INFO:root:(('127.0.0.1', 55927)) Waiting other nodes.
INFO:root:(('127.0.0.1', 48497)) Evaluating...
INFO:root:(('127.0.0.1', 52265)) Round 2 of 2 finished.
INFO:root:(('127.0.0.1', 51551)) Training...
INFO:root:(('127.0.0.1', 55927)) Round 2 of 2 finished.
INFO:root:(('127.0.0.1', 48497)) Training...
INFO:root:(('127.0.0.1', 52265)) Evaluating...
INFO:root:(127.0.0.1:51551) Model added (1/2) from ('127.0.0.1', 51551)
INFO:root:(('127.0.0.1', 55927)) Evaluating...
INFO:root:(('127.0.0.1', 52265)) Finish!!.
INFO:root:(127.0.0.1:48497) Model added (1/2) from ('127.0.0.1', 48497)
INFO:root:(('127.0.0.1', 55927)) Finish!!.
INFO:root:(('127.0.0.1', 51551)) Broadcasting model to 3 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 48497)) Broadcasting model to 3 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 48497)) Waiting other nodes.
INFO:root:(('127.0.0.1', 48497)) Broadcasting model to 3 clients. (size: 548864 bytes)
ERROR:root:

In [1]:
('127.0.0.1', 50827) in [('127.0.0.1', 54975), ('127.0.0.1', 50827), ('127.0.0.1', 34699), ('127.0.0.1', 46939)]


True

In [1]:
('127.0.0.1', 47735) in [('127.0.0.1', 47735), ('127.0.0.1', 41705), ('127.0.0.1', 53983), ('127.0.0.1', 52883)]

True

In [3]:
a = [('127.0.0.1', 51413), ('127.0.0.1', 50463), ('127.0.0.1', 49439), ('127.0.0.1', 55467)]

In [4]:
[str(x) for x in a]

["('127.0.0.1', 51413)",
 "('127.0.0.1', 50463)",
 "('127.0.0.1', 49439)",
 "('127.0.0.1', 55467)"]